<a href="https://colab.research.google.com/github/ATHARVA316-DEV/-Eye-Cataract-Detection-1/blob/main/try4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet timm albumentations opencv-python-headless matplotlib pandas scikit-learn torchmetrics openpyxl
import sys, importlib
packages = ["torch", "timm", "albumentations", "cv2", "matplotlib", "pandas", "sklearn"]
for p in packages:
    try:
        mod = importlib.import_module(p if p!="cv2" else "cv2")
        print(p, getattr(mod, "__version__", "version-ok"))
    except Exception as e:
        print(p, "import error:", e)

torch 2.8.0+cu126
timm 1.0.21
albumentations 2.0.8
cv2 4.12.0
matplotlib 3.10.0
pandas 2.2.2
sklearn 1.6.1


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/ocular-disease-recognition-odir5k")

print("Path to dataset files:", path)

100%|██████████| 1.62G/1.62G [01:17<00:00, 22.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2


In [ ]:
import os
path="/root/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2"
base_path = path  # ← use the same variable, do NOT rewrite manually

# Show folder structure
for root, dirs, files in os.walk(base_path):
    print(root)
    break

!ls -R "$base_path"


/root/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2
/root/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2:
full_df.csv  ODIR-5K  preprocessed_images

/root/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2/ODIR-5K:
ODIR-5K

/root/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2/ODIR-5K/ODIR-5K:
 data.xlsx  'Testing Images'  'Training Images'

'/root/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2/ODIR-5K/ODIR-5K/Testing Images':
1000_left.jpg	1437_left.jpg	2260_left.jpg	3532_left.jpg	4665_left.jpg
1000_right.jpg	1437_right.jpg	2260_right.jpg	3532_right.jpg	4665_right.jpg
1001_left.jpg	1438_left.jpg	2261_left.jpg	3533_left.jpg	4666_left.jpg
1001_right.jpg	1438_right.jpg	2261_right.jpg	3533_right.jpg	4666_right.jpg
1002_left.jpg	1445_left.jpg	2263_left.jpg	3534_left.jpg	4667_left.jpg
1002_right.jpg	1445_right.jpg	2263_right.jp

In [ ]:
import os

base_path = os.path.join(path, "ODIR-5K", "ODIR-5K")

labels_file = os.path.join(base_path, "data.xlsx")
train_images_folder = os.path.join(base_path, "Training Images")
test_images_folder = os.path.join(base_path, "Testing Images")



In [ ]:
import pandas as pd

df = pd.read_excel(labels_file)
print(df.head())



   ID  Patient Age Patient Sex Left-Fundus Right-Fundus  \
0   0           69      Female  0_left.jpg  0_right.jpg   
1   1           57        Male  1_left.jpg  1_right.jpg   
2   2           42        Male  2_left.jpg  2_right.jpg   
3   3           66        Male  3_left.jpg  3_right.jpg   
4   4           53        Male  4_left.jpg  4_right.jpg   

                            Left-Diagnostic Keywords  \
0                                           cataract   
1                                      normal fundus   
2  laser spot，moderate non proliferative retinopathy   
3                                      normal fundus   
4                        macular epiretinal membrane   

                Right-Diagnostic Keywords  N  D  G  C  A  H  M  O  
0                           normal fundus  0  0  0  1  0  0  0  0  
1                           normal fundus  1  0  0  0  0  0  0  0  
2  moderate non proliferative retinopathy  0  1  0  0  0  0  0  1  
3         branch retinal artery occl